# 카카오API 위도경도 받아오기

In [1]:
import requests, json
import pandas as pd
from urllib.parse import quote

In [2]:
#API 키 가져오기
with open('KakaoAPI.txt') as file:
    road_key = file.read()

In [5]:
# URL 만들기
base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
addr = '서울특별시 서대문구 신촌역로 4 (대현동)'
url = f'{base_url}?query={quote(addr)}'

In [6]:
#Header : Authorization: KakaoAK ${REST_API_KEY}
header = {'Authorization': f'KakaoAK {road_key}'}

# Kakao Local API 호출하여 결과를 가져오기

In [7]:
result = requests.get(url, headers=header).json()
result

{'documents': [{'address': {'address_name': '서울 서대문구 대현동 101-22',
    'b_code': '1141011200',
    'h_code': '1141058500',
    'main_address_no': '101',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '서대문구',
    'region_3depth_h_name': '신촌동',
    'region_3depth_name': '대현동',
    'sub_address_no': '22',
    'x': '126.94311045086',
    'y': '37.5571114864975'},
   'address_name': '서울 서대문구 신촌역로 4',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 서대문구 신촌역로 4',
    'building_name': '',
    'main_building_no': '4',
    'region_1depth_name': '서울',
    'region_2depth_name': '서대문구',
    'region_3depth_name': '대현동',
    'road_name': '신촌역로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.94311045086',
    'y': '37.5571114864975',
    'zone_no': '03766'},
   'x': '126.94311045086',
   'y': '37.5571114864975'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [8]:
lat = result['documents'][0]['y']
lng = result['documents'][0]['x']

In [9]:
lat, lng

('37.5571114864975', '126.94311045086')

# 지하철역 정보 완성하기

In [10]:
df = pd.read_csv('test_data/신촌주변지하철역.csv')
df

,이름,주소
0,신촌역,서울특별시 서대문구 신촌역로 4 (대현동)
1,홍대입구역,서울특별시 마포구 양화로 지하160 (동교동)
2,여의도역,서울특별시 영등포구 여의나루로 지하40 (여의도동)
3,서대문구청,서울특별시 서대문구 연희로 247 (연희동)


In [14]:
lat_list, lng_list = [], []
for i in df.index:
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json()
    lat_list.append(result['documents'][0]['y'])
    lng_list.append(result['documents'][0]['x'])

In [15]:
lat_list

['37.5571114864975',
 '37.5568904093718',
 '37.5218423045786',
 '37.5791549275777']

In [16]:
lng_list

['126.94311045086', '126.923674307594', '126.924484687694', '126.935815915686']

In [17]:
df['위도'] = lat_list
df['경도'] = lng_list

In [18]:
df

,이름,주소,위도,경도
0,신촌역,서울특별시 서대문구 신촌역로 4 (대현동),37.5571114864975,126.94311045086
1,홍대입구역,서울특별시 마포구 양화로 지하160 (동교동),37.5568904093718,126.923674307594
2,여의도역,서울특별시 영등포구 여의나루로 지하40 (여의도동),37.5218423045786,126.924484687694
3,서대문구청,서울특별시 서대문구 연희로 247 (연희동),37.5791549275777,126.935815915686


In [19]:
df.to_csv('test_data/신촌주변지하철역.csv', index=False)

In [20]:
pd.read_csv('test_data/신촌주변지하철역.csv')

,이름,주소,위도,경도
0,신촌역,서울특별시 서대문구 신촌역로 4 (대현동),37.557111,126.943110
1,홍대입구역,서울특별시 마포구 양화로 지하160 (동교동),37.556890,126.923674
2,여의도역,서울특별시 영등포구 여의나루로 지하40 (여의도동),37.521842,126.924485
3,서대문구청,서울특별시 서대문구 연희로 247 (연희동),37.579155,126.935816
